In [130]:
using DataFrames
using JSON
using Iterators
using taxis
using HDF5, JLD
using Stats
using kNN
using sequenceCompare
#reload("taxis")
#reload("sequenceCompare")
nprocs()

8

In [131]:
taxi_df, taxi_validation_df = taxis.LoadData("/home/tony/ML/taxi/taxi2_time/train_200k.csv", "/home/tony/ML/taxi/taxi2_time/test.csv")
0

Begin
loading csv files
loading coords
getting coords counts
deleting unneeded data rows/columns
done!


0

In [133]:
train_coords = taxi_df[:COORDS_TEST][1:4]
#train_lens = [c[:,1:int(ceil(rand(1)[1]*size(c,2)))] for c in train_coords]

4-element DataArray{Any,1}:
 2x9 Array{Float64,2}:
 -8.61864  -8.6185  -8.62033  -8.62215  …  -8.62737  -8.63023  -8.63275
 41.1414   41.1414  41.1425   41.1438      41.1447   41.1452   41.1469    
 2x11 Array{Float64,2}:
 -8.63985  -8.64035  -8.6422  -8.64446  …  -8.66018  -8.66311  -8.66624
 41.1598   41.1599   41.1601  41.1605      41.1632   41.1637   41.1642   
 2x57 Array{Float64,2}:
 -8.61296  -8.61338  -8.61421  -8.61477  …  -8.63534  -8.63328  -8.63151
 41.1404   41.1403   41.1403   41.1404      41.148    41.1472   41.1461 
 2x31 Array{Float64,2}:
 -8.57468  -8.5747  -8.5747  -8.57466  …  -8.59882  -8.60019  -8.60106
 41.152    41.1519  41.1519  41.152       41.1461   41.1462   41.1461     

In [240]:
println("looking at taxi id information")

head(taxi_df)
taxi_ids = taxi_df[:TAXI_ID]
taxi_ids_dict = Dict{Int64, Int64}()
for id in taxi_ids
    taxi_ids_dict[id] = get(taxi_ids_dict, id, 0) + 1 
end

taxi_id_counts = [x::Int64 for x in values(taxi_ids_dict)]
describe(taxi_id_counts)
println("number less than 10: ", sum(taxi_id_counts .< 20)) 
println("number of taxi ids: ", length(keys(taxi_ids_dict)))

#taxis_by_id = [id => taxi_df[taxi_df[:TAXI_ID].==id,:] for id in keys(taxi_ids_dict)]

val_ids = [get(taxi_ids_dict, x, 0)::Int64 for x in taxi_validation_df[:TAXI_ID]]
sum(val_ids .== 0)

looking at taxi id information
Summary Stats:
Mean:         463.465116
Minimum:      1.000000
1st Quartile: 329.250000
Median:       428.500000
3rd Quartile: 607.000000
Maximum:      1067.000000
number less than 10: 3
number of taxi ids: 430


6

In [104]:
println("finding unique number of coords")
all_coords_val = hcat(taxi_validation_df[:COORDS]...)
all_coords = hcat(taxi_df[:COORDS]...)

finding unique number of coords


2x4774976 Array{Float64,2}:
 -8.61864  -8.6185  -8.62033  -8.62215  …  -8.6304  -8.63042  -8.63041
 41.1414   41.1414  41.1425   41.1438      41.1579  41.158    41.1579 

# Creating coord dict

In [53]:
small_taxi_df = GetTableOrderedSubset(taxi_df, 190000)
coordsDB = ConstructCoordsDatabase(small_taxi_df, 2)
describe([length(x)::Int64 for x in values(coordsDB)])

190000, 190000Summary Stats:
Mean:         1616.671036
Minimum:      1.000000
1st Quartile: 4.000000
Median:       10.000000
3rd Quartile: 37.000000
Maximum:      963095.000000


In [63]:
taxi_df[:GRID_START] = [round(c,2) for c in taxi_df[:START]]
taxi_validation_df[:GRID_START] = [round(c,2) for c in taxi_validation_df[:START]]
taxi_df[:GRID_END] = [round(c,2) for c in taxi_df[:END]]
taxi_validation_df[:GRID_END] = [round(c,2) for c in taxi_validation_df[:END]]


320-element Array{Any,1}:
 [-8.58,41.15]
 [-8.6,41.16] 
 [-8.57,41.17]
 [-8.61,41.14]
 [-8.62,41.15]
 [-8.63,41.17]
 [-8.58,41.15]
 [-8.58,41.19]
 [-8.61,41.13]
 [-8.59,41.2] 
 [-8.59,41.17]
 [-8.6,41.15] 
 [-8.58,41.16]
 ⋮            
 [-8.59,41.15]
 [-8.64,41.14]
 [-8.68,41.15]
 [-8.68,41.17]
 [-8.59,41.15]
 [-8.53,41.14]
 [-8.6,41.2]  
 [-8.59,41.15]
 [-8.63,41.15]
 [-8.64,41.15]
 [-8.57,41.17]
 [-8.59,41.19]

In [128]:

function getGridDicts(train_df, test_df, grid_delta=.01)

    train_df[:GRID_START] = [round(c,2) for c in train_df[:START]]
    test_df[:GRID_START]  = [round(c,2) for c in test_df[:START] ]
    train_df[:GRID_END]   = [round(c,2) for c in train_df[:END]  ]
    test_df[:GRID_END]    = [round(c,2) for c in test_df[:END]   ]
    grids_dict = Dict()

    for i in 1:length(train_df[:GRID_START])
        coord_pair = train_df[:GRID_START][i]
        res = get(grids_dict, coord_pair, Any[])
        push!(res, train_df[:COORDS][i])
        grids_dict[coord_pair] = res 
    end

    mean_length_dict = Dict()
    mean_dest_dict = Dict()

    for grid_coord in keys(grids_dict)
        paths = grids_dict[grid_coord]
        endpoints = [x[:,end] for x in paths]
        path_lens = [int(length(x)/2)::Int64 for x in paths]
        #println("hey")
        mean_dest_dict[grid_coord] = mean(endpoints)
        #println("2222")
        mean_length_dict[grid_coord] = round(mean(path_lens))
    end

    return mean_length_dict, mean_dest_dict    

end

mean_length_dict, mean_dest_dict = getGridDicts()


In [129]:
values(mean_length_dict)

ValueIterator for a Dict{Any,Any} with 419 entries. Values:
  63.0
  45.0
  54.0
  10.0
  44.0
  33.0
  61.0
  76.0
  44.0
  88.0
  129.0
  79.0
  48.0
  31.0
  53.0
  52.0
  55.0
  50.0
  54.0
  58.0
  71.0
  132.0
  391.0
  67.0
  ⋮

In [122]:
pairs = collect(keys(grids_dict))
grid_coord = pairs[1]

p = grids_dict[grid_coord]
println("num paths: ", length(p))
endpoints = [x[:,end] for x in p]
paths_lens = [int(length(x)/2)::Int64 for x in p]
println("endpoiktns and paths")
println(mean(paths_lens))

println(length(p[1]))

for x in p
    println(length(x)) 
end
println(mean(endpoints))


num paths: 6
endpoiktns and paths
99646.0
199292
199292
199292
199292
199292
199292
199292


LoadError: `convert` has no method matching convert(::Type{Float64}, ::Array{Float64,2})
while loading In[122], in expression starting on line 16

In [90]:
sum([haskey(grids_dict, x) for x in taxi_validation_df[:GRID_START]])
mean([c for c in train_df[:END][1:5]])
median(train_df[:NUM_COORDS][1:5])

29.0

#Creating new features


In [ ]:
function GetDateInfo(df)
    if haskey(df, :DAYOFWEEK)
        return df
end

function GetDistanceData(df)
    if haskey(df, :DISTANCE)
        return df
end

# Training models

In [48]:
function euclideanDist(p1, p2)
    return sqrt((p1[1]-p2[1])^2 + (p1[2]-p2[2])^2)
end

function findClosestTrainingExample2(all_train_coords, test_path, w=1)
    num_paths = length(all_train_coords)
    best_dist = 9999.0
    best_path = all_train_coords[1]
    for k=1:num_paths
        train_path = all_train_coords[k]
        if w <= 1
            dist = DTWDistance(train_path, test_path) 
        else
            dist = DTWDistance(train_path, test_path, w) 
        end
        #println("k=", k, " dist=", dist)
        if dist < best_dist
            #println(k, ", old best: ", best_dist, " new best: ", dist)
            best_dist = dist
            best_path = all_train_coords[k]
        end
    end
    
    return best_path
end

# Create a DTW/START_DIFF metric
function findClosestTrainingExample3(all_train_coords, test_path, w=1)
    num_paths = length(all_train_coords)
    best_score = 999999.0
    best_path = all_train_coords[1]
    for k=1:num_paths
        
        train_path = all_train_coords[k]
        test_path = test_path[:,1:min(end, size(train_path,2)+2)]
        
        if size(train_path,2) < size(test_path, 2)
            continue
        end
        
        if w <= 1
            dist = DTWDistance(train_path, test_path) 
        else
            dist = DTWDistance(train_path, test_path, w) 
        end
        
        start_diff = float(euclideanDist(test_path[:,1], train_path[:,1])+0.000001)
        
        # score is 1/dist * 1/start_diff = 1/(dist*start_diff)
        # we'll minimize score = (dist*start_diff)
        score = float(dist * start_diff)
        
        if score < best_score
            best_score = score
            best_path = all_train_coords[k]
        end
    end
    
    return best_path
end

function findClosestTrainingExamplesDTWstarDiffAvg(all_train_coords, test_path, w=2, num_avg=30)
    num_paths = length(all_train_coords)
    
    dists = [float(DTWDistance(train_path, test_path, w)) for train_path in all_train_coords]
    start_diffs = [float(euclideanDist(test_path[:,1], train_path[:,1])+0.0000001) for train_path in all_train_coords]
    num_coords = [size(train_path,2) for train_path in all_train_coords]
    
    
    df = DataFrame(DISTS = dists, START_DIFF = start_diffs, NUM_COORDS = num_coords, DTW_START_DIFF = dists .* start_diffs)
    sort!(df, cols=[:DTW_START_DIFF])
    avg_length = median(df[:NUM_COORDS][1:min(end,num_avg)])
    
    return int(avg_length)
end


function findClosestTrainingExampleForTestSet2(train_df, test_df, max_subset=1000, w=1)
    all_train_paths = train_df[:COORDS]
    all_test_paths  = test_df[:COORDS]
    num_train_paths = length(all_train_paths)
    num_test_paths = length(all_test_paths)
    train_paths_subset = all_train_paths[7000:(7000+max_subset)]
    closest_examples = cell(num_test_paths)
    
    for k=1:num_test_paths
        if k % 20 == 0
            println(k, "/", num_test_paths, " for ", num_train_paths, " train path examples") 
        end
        test_path = all_test_paths[k]
        test_path_len = size(test_path, 2)
        test_taxi_id = test_df[:TAXI_ID][k]
        
        same_taxi_df = train_df[train_df[:TAXI_ID] .== test_taxi_id,:]
        if size(same_taxi_df,1) == 0        
            println("no taxi ids found for ", test_taxi_id)
            closest_training_example = findClosestTrainingExample(train_paths_subset, test_path, w) 
            closest_examples[k] = closest_training_example
        else 
            #println(size(same_taxi_df,1), "    - number of taxi id's routes")
            #println("test path length: ", test_path_len)
            #println("taxi id: ", test_taxi_id)
            same_taxi_paths = same_taxi_df[same_taxi_df[:NUM_COORDS] .>= test_path_len,:][:COORDS][1:min(end,max_subset)]
            println(size(same_taxi_df,1), "    - number of taxi id's routes with ", length(same_taxi_paths),
            " greater than length ", test_path_len,"!")

            if length(same_taxi_paths) == 0
                println("all paths filtered out!  resorting to random search again!")
                closest_training_example = findClosestTrainingExample(train_paths_subset, test_path, w) 
                closest_examples[k] = closest_training_example
            else
                closest_training_example = findClosestTrainingExample(same_taxi_paths, test_path, w) 
                closest_examples[k] = closest_training_example                
            end
        end
    end
    
    return closest_examples
end

function findClosestTrainingExampleForTestSet3(train_df, test_df, max_subset=4000, w=1)
    all_train_paths = train_df[:COORDS]
    all_test_paths  = test_df[:COORDS]
    num_train_paths = length(all_train_paths)
    num_test_paths = length(all_test_paths)
    train_paths_subset = all_train_paths[7000:(7000+max_subset)]
    closest_examples = cell(num_test_paths)
    
    for k=1:num_test_paths
        if k % 20 == 0
            println(k, "/", num_test_paths, " for ", num_train_paths, " train path examples") 
        end
        test_path = all_test_paths[k]
        test_path_len = size(test_path, 2)
        test_taxi_id = test_df[:TAXI_ID][k]
        
        same_taxi_df = train_df[train_df[:TAXI_ID] .== test_taxi_id,:]
        if size(same_taxi_df,1) == 0        
            println("no taxi ids found for ", test_taxi_id)
            closest_training_example = findClosestTrainingExample3(train_paths_subset, test_path, w) 
            closest_examples[k] = closest_training_example
        else 
            #println(size(same_taxi_df,1), "    - number of taxi id's routes")
            #println("test path length: ", test_path_len)
            #println("taxi id: ", test_taxi_id)
            same_taxi_paths = same_taxi_df[same_taxi_df[:NUM_COORDS] .>= test_path_len,:][:COORDS]
            println(size(same_taxi_df,1), "    - number of taxi id's routes with ", length(same_taxi_paths),
            " greater than length ", test_path_len,"!")

            if length(same_taxi_paths) == 0
                println("all paths filtered out!  resorting to random search again!")
                closest_training_example = findClosestTrainingExample3(train_paths_subset, test_path, w) 
                closest_examples[k] = closest_training_example
            else
                closest_training_example = findClosestTrainingExample3(same_taxi_paths, test_path, w) 
                closest_examples[k] = closest_training_example                
            end
        end
    end
    
    return closest_examples
end

function avgDTWstartDiffScore(train_df, test_df, max_subset=4000, w=2, num_avg=30)
    all_train_paths = train_df[:COORDS]
    all_test_paths  = test_df[:COORDS]
    num_train_paths = length(all_train_paths)
    num_test_paths = length(all_test_paths)
    train_paths_subset = all_train_paths[7000:(7000+max_subset)]
    closest_examples = cell(num_test_paths)
    
    for k=1:num_test_paths
        if k % 20 == 0
            println(k, "/", num_test_paths, " for ", num_train_paths, " train path examples") 
        end
        test_path = all_test_paths[k]
        test_path_len = size(test_path, 2)
        test_taxi_id = test_df[:TAXI_ID][k]
        
        same_taxi_df = train_df[train_df[:TAXI_ID] .== test_taxi_id,:]
        if size(same_taxi_df,1) == 0        
            println("no taxi ids found for ", test_taxi_id)
            closest_training_example = findClosestTrainingExamplesDTWstarDiffAvg(train_paths_subset, test_path, w, num_avg) 
            closest_examples[k] = closest_training_example
        else 
            #println(size(same_taxi_df,1), "    - number of taxi id's routes")
            #println("test path length: ", test_path_len)
            #println("taxi id: ", test_taxi_id)
            same_taxi_paths = same_taxi_df[same_taxi_df[:NUM_COORDS] .>= test_path_len,:][:COORDS]
            println(size(same_taxi_df,1), "    - number of taxi id's routes with ", length(same_taxi_paths),
            " greater than length ", test_path_len,"!")

            if length(same_taxi_paths) == 0
                println("all paths filtered out!  resorting to random search again!")
                closest_training_example = findClosestTrainingExamplesDTWstarDiffAvg(train_paths_subset, test_path, w, num_avg) 
                closest_examples[k] = closest_training_example
            else
                closest_training_example = findClosestTrainingExamplesDTWstarDiffAvg(same_taxi_paths, test_path, w, num_avg) 
                closest_examples[k] = closest_training_example   
            end
        end
    end
    
    return closest_examples
end

avgDTWstartDiffScore (generic function with 4 methods)

In [ ]:
function startEndPrediction(train_paths, test_paths)
    num_test_examples = length(test_paths)
    
end

In [16]:
println("SUBMISSION PREDICTION")

# 1. Using subset of all_train_coords and scanning it for each test example

#all_train_coords = taxi_df[:COORDS][1:20]
#all_validation_coords = taxi_validation_df[:COORDS]
#test_guess_paths = findClosestTrainingExampleForTestSet(all_train_coords, all_validation_coords)
#taxi_validation_df[:GUESS_PATHS] = test_guess_paths

# 2. Scanning paths of same taxi


all_train_coords = taxi_df[:COORDS]
all_validation_coords = taxi_validation_df[:COORDS]
test_guess_lengths = avgDTWstartDiffScore(taxi_df, taxi_validation_df, 4000, 2, 30)
taxi_validation_df[:GUESS_NUM_COORDS] = test_guess_lengths


SUBMISSION PREDICTION
744    - number of taxi id's routes with 701 greater than length 11!
820    - number of taxi id's routes with 366 greater than length 40!
602    - number of taxi id's routes with 231 greater than length 40!
847    - number of taxi id's routes with 830 greater than length 8!
899    - number of taxi id's routes with 897 greater than length 2!
437    - number of taxi id's routes with 53 greater than length 137!
794    - number of taxi id's routes with 684 greater than length 24!
367    - number of taxi id's routes with 345 greater than length 17!
706    - number of taxi id's routes with 245 greater than length 43!
309    - number of taxi id's routes with 27 greater than length 79!
560    - number of taxi id's routes with 100 greater than length 63!
595    - number of taxi id's routes with 567 greater than length 17!
368    - number of taxi id's routes with 366 greater than length 5!
103    - number of taxi id's routes with 46 greater than length 76!
608    - number o

320-element Array{Any,1}:
 2x12 Array{Float64,2}:
 -8.58567  -8.58601  -8.58618  -8.58641  …  -8.57888  -8.57812  -8.57977
 41.1486   41.149    41.1488   41.1477      41.1457   41.1445   41.1439  
 2x61 Array{Float64,2}:
 -8.61078  -8.61078  -8.61034  -8.61022  …  -8.60307  -8.60139  -8.60044
 41.1457   41.1458   41.1462   41.1463      41.1836   41.1832   41.1829  
 2x57 Array{Float64,2}:
 -8.58567  -8.58589  -8.58612  -8.58651  …  -8.55481  -8.55429  -8.55346
 41.1485   41.149    41.1487   41.1477      41.1812   41.1811   41.1811  
 2x16 Array{Float64,2}:
 -8.61394  -8.6147  -8.6144  -8.61492  …  -8.61522  -8.61518  -8.61513
 41.1412   41.1415  41.1425  41.1428      41.1467   41.1467   41.1467      
 2x11 Array{Float64,2}:
 -8.61994  -8.61994  -8.61993  -8.62003  …  -8.61973  -8.6189  -8.61873
 41.148    41.148    41.148    41.1479      41.1524   41.1522  41.1523    
 2x141 Array{Float64,2}:
 -8.64465  -8.64515  -8.64402  -8.64381  …  -8.64481  -8.64479  -8.64475
 41.1577   41.1583   

In [50]:
println("Local prediction")

#scanning subset of train paths for all test paths

#all_train_coords = taxi_df[:COORDS][10000:10400]
#test_df = tail(taxi_df, 100)
#all_test_coords = test_df[:COORDS_TEST]
#test_guess_paths = findClosestTrainingExampleForTestSet(all_train_coords, all_test_coords, 2)
#test_df[:GUESS_PATHS] = test_guess_paths

#looking at paths for same driver
train_df = taxi_df[1:90000,:]
test_df = tail(taxi_df, 10)
test_coords = test_df[:COORDS]
test_df[:COORDS] = test_df[:COORDS_TEST]
test_guess_lengths = avgDTWstartDiffScore(train_df, test_df, 1000, 2, 140)
test_df[:GUESS_NUM_COORDS] = test_guess_lengths
test_df[:COORDS] = test_coords #change it back for scoring

score_coords_guess(test_df)

Local prediction


LoadError: avgDTWstartDiffScore not defined
while loading In[50], in expression starting on line 16

In [52]:


function score_path_guess(test_df)
    pred_paths = test_df[:GUESS_PATHS]
    actual_paths = test_df[:COORDS]
    pred_times = [((size(x,2)-1)*15)::Int64 for x in pred_paths]
    actual_times = [((size(x,2)-1)*15)::Int64 for x in actual_paths]
    score = sqrt(mean((log(max(pred_times, 660)+1)-log(actual_times+1)).^2))
    println("time score: ", score) 
    for k=1:length(pred_times)
        println("pred: ", pred_times[k], ", actual: ", actual_times[k], ", delta error: ", pred_times[k]-actual_times[k])
    end
end

function score_coords_guess(test_df)
    pred_num_coords = test_df[:GUESS_NUM_COORDS]
    actual_paths = test_df[:COORDS]
    pred_times = [((x-1)*15)::Int64 for x in pred_num_coords]
    actual_times = [((size(x,2)-1)*15)::Int64 for x in actual_paths]
    score = sqrt(mean((log(max(pred_times, 660)+1)-log(actual_times+1)).^2))
    println("time score: ", score) 
    for k=1:length(pred_times)
        println("pred: ", pred_times[k], ", actual: ", actual_times[k], ", delta error: ", pred_times[k]-actual_times[k])
    end
end


score_path_guess(test_df)


time score: 1.574925625091306
pred: 900, actual: 660, delta error: 240
pred: 1560, actual: 1005, delta error: 555
pred: 0, actual: 0, delta error: 0
pred: 345, actual: 735, delta error: -390
pred: 240, actual: 360, delta error: -120
pred: 255, actual: 1065, delta error: -810
pred: 465, actual: 465, delta error: 0
pred: 960, actual: 1005, delta error: -45
pred: 420, actual: 540, delta error: -120
pred: 585, actual: 360, delta error: 225
pred: 465, actual: 615, delta error: -150
pred: 255, actual: 480, delta error: -225
pred: 735, actual: 1080, delta error: -345
pred: 1065, actual: 1005, delta error: 60
pred: 870, actual: 840, delta error: 30
pred: 30, actual: 75, delta error: -45
pred: 975, actual: 945, delta error: 30
pred: 240, actual: 255, delta error: -15
pred: 435, actual: 450, delta error: -15
pred: 540, actual: 555, delta error: -15


In [126]:
log([1 2 3])

1x3 Array{Float64,2}:
 0.0  0.693147  1.09861

In [17]:
#taxi_validation_df[:GUESS_PATHS] = test_guess_paths

#guess_times = [length(x)*15 for x in test_guess_paths]
num_test_examples = length(test_guess_paths)

guess_times = Array(Int64, num_test_examples)
dest_coords = cell(num_test_examples)

all_test_paths = taxi_validation_df[:COORDS]
test_guess_paths = taxi_validation_df[:GUESS_PATHS]
for k=1:num_test_examples
    test_path = all_test_paths[k]
    best_guess_path = test_guess_paths[k]
    
    test_path_time = length(test_path)*15
    best_guess_time = length(best_guess_path)*15
    
    if test_path_time > best_guess_time
        println(k, ":  guessing ", best_guess_time, " but existing time is ", test_path_time)
        #best_guess_time = test_path_time + 100
    end
    guess_times[k] = max(test_path_time, best_guess_time)
end

submission_validation = guess_times

121:  guessing 990 but existing time is 1080
126:  guessing 8220 but existing time is 11610
152:  guessing 30 but existing time is 300
167:  guessing 3450 but existing time is 11070
186:  guessing 30 but existing time is 60
199:  guessing 1200 but existing time is 9810
201:  guessing 1350 but existing time is 18360
214:  guessing 390 but existing time is 510


18360

In [174]:
size(taxi_df[:COORDS][1],2)

23

#Generating Submission

In [18]:
# beat the benchmark example
#mean_time = mean(times_validation)
#submission_validation = [max(x, mean_time) for x in times_validation]

#submission_validation

df_submission = DataFrame()
df_submission[:TRIP_ID] = taxi_validation_df[:TRIP_ID]
df_submission[:TRAVEL_TIME] = submission_validation
writetable("7th_submission_DTW_START_DIFF.csv", df_submission)

In [27]:
lats = [float(x[2,end])::Float64 for x in taxi_validation_df[:GUESS_PATHS]]
lons = [float(x[1,end])::Float64 for x in taxi_validation_df[:GUESS_PATHS]]

df_submission = DataFrame()
df_submission[:TRIP_ID] = taxi_validation_df[:TRIP_ID]
df_submission[:LATITUDE] = lats
df_submission[:LONGITUDE] = lons
writetable("1th_sub_endpoint_DTW_START_DIFF.csv", df_submission)

In [388]:
immutable Point2{T}
  x::T
  y::T
end

D = [Point2(1.,2.) => 42]
haskey(D, Point2(1., 2.))  #False!


true

LoadError: ArgumentError("setindex!(::DataFrame, ...) only broadcasts scalars, not arrays")
while loading In[42], in expression starting on line 2

In [39]:
taxi_validation_df[:COORDS]

320-element DataArray{Any,1}:
 2x11 Array{Float64,2}:
 -8.58568  -8.58571  -8.58568  -8.58573  …  -8.587   -8.58658  -8.58488
 41.1485   41.1486   41.1489   41.1489      41.1475  41.1472   41.1466    
 2x40 Array{Float64,2}:
 -8.61088  -8.61086  -8.6109  -8.61044  …  -8.60293  -8.60255  -8.60189
 41.1456   41.1456   41.1458  41.1462      41.1628   41.1631   41.1636    
 2x40 Array{Float64,2}:
 -8.58574  -8.58573  -8.58572  -8.58629  …  -8.57695  -8.5759  -8.5749
 41.1486   41.1488   41.149    41.149       41.1664   41.1672  41.1677     
 2x8 Array{Float64,2}:
 -8.61396  -8.61412  -8.61509  -8.61528  …  -8.61524  -8.61505  -8.61464
 41.1412   41.1411   41.1409   41.1408      41.1408   41.1408   41.141    
 2x2 Array{Float64,2}:
 -8.6199  -8.61989
 41.148   41.148                                                                                                                
 2x137 Array{Float64,2}:
 -8.63061  -8.63061  -8.63074  -8.63151  …  -8.62639  -8.6264  -8.62641
 41.1782   41.1782